In [1]:
!pip install tensorflow opencv-python pillow


In [8]:
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
from IPython.display import display, clear_output


In [10]:
cascade_path = cv2.data.haarcascades + "haarcascade_trafficlight.xml"

cascade_path

'D:\\Anaconda\\Lib\\site-packages\\cv2\\data\\haarcascade_trafficlight.xml'

In [1]:


import cv2
import tensorflow as tf
import numpy as np
from PIL import Image
from IPython.display import display, clear_output

# Load the trained CNN model
model_path = "my_cnn_model_v9.keras"
model = tf.keras.models.load_model(model_path)
print("Model Loaded and Ready for Live Predictions!")

# Class labels
class_labels = {
    0: 'GO',
    1: 'GO LEFT',
    2: 'SLOW',
    3: 'SLOW LEFT',
    4: 'STOP',
    5: 'STOP LEFT'
}


cascade_path = "cascade_haar.xml"
traffic_light_cascade = cv2.CascadeClassifier(cascade_path)


if traffic_light_cascade.empty():
    print("Error: Haar cascade file not loaded. Check the file path.")
    exit()

# Function to preprocess the frame for the model
def preprocess_image(frame, target_size=(64, 64)):
    frame_resized = cv2.resize(frame, target_size)
    frame_array = frame_resized / 255.0  # Normalize pixel values
    return tf.expand_dims(frame_array, axis=0)  # Add batch dimension

# Function to predict the class of the frame
def predict_image(model, preprocessed_image):
    prediction = model.predict(preprocessed_image)
    predicted_class = tf.argmax(prediction, axis=1).numpy()[0]
    confidence = tf.reduce_max(prediction).numpy()
    return predicted_class, confidence

# Function for live detection and classification
def live_class_detection_in_notebook():
    cap = cv2.VideoCapture(0)  # Open the webcam
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return

    print("\nStarting live class detection with bounding boxes. Press 'q' to quit.\n")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame from the camera.")
                break

            # Convert frame to grayscale for Haar cascade
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect traffic lights using Haar cascade
            traffic_lights = traffic_light_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in traffic_lights:
           
                roi = frame[y:y+h, x:x+w]

                # Preprocess ROI for the classification model
                preprocessed_image = preprocess_image(roi)

                # Predict the class of the detected object
                predicted_class, confidence = predict_image(model, preprocessed_image)
                predicted_label = class_labels.get(predicted_class, 'Unknown')

                # Draw bounding box and label on the frame
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                label = f"{predicted_label} ({confidence:.2f})"
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # Convert frame to RGB (from BGR used by OpenCV) for display in Jupyter
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame_rgb)

            # Display the frame in Jupyter Notebook
            clear_output(wait=True)  # Clear the previous frame
            display(img)

    except KeyboardInterrupt:
        print("\nLive detection interrupted by user.")

    finally:
        cap.release()
        print("\nLive class detection ended.")
 

# Start live detection in Jupyter Notebook
live_class_detection_in_notebook()



ImportError: initialization failed

In [13]:
import tensorflow as tf
print(tf.__version__)


2.17.0
